# Artificial Neural Networks (ANN)

## Objectives

```
- Classification of data with multi-class labels.
- 
- 
```

In [1]:
# Built-in library
from typing import Any, Optional, Sequence, Union

# Standard imports
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
# Configure the backend
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
import seaborn as sns

# load data
iris_data = sns.load_dataset("iris")

# Preprocess the data
condlist = [
    (iris_data["species"] == "setosa"),
    (iris_data["species"] == "versicolor"),
    iris_data["species"] == "virginica",
]
choicelist = [0, 1, 2]
iris_data["target"] = np.select(condlist=condlist, choicelist=choicelist)

iris_data.sample(n=5, random_state=1)

,sepal_length,sepal_width,petal_length,petal_width,species,target
14,5.8,4.0,1.2,0.2,setosa,0
98,5.1,2.5,3.0,1.1,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
16,5.4,3.9,1.3,0.4,setosa,0
131,7.9,3.8,6.4,2.0,virginica,2


In [3]:
iris_data.get("species").value_counts(normalize=True)

species
setosa        0.333333
versicolor    0.333333
virginica     0.333333
Name: proportion, dtype: float64

In [4]:
# Convert the data to Torch tensor
X = torch.tensor(iris_data.loc[:, iris_data.columns[:4]].values).float()
y = torch.tensor(iris_data["target"].values).long()

X.shape, y.shape

(torch.Size([150, 4]), torch.Size([150]))

In [5]:
# Build model
clf = nn.Sequential(
    # input
    nn.Linear(4, 64),
    # Activation
    nn.ReLU(),
    # Hidden
    nn.Linear(64, 64),
    # Activation
    nn.ReLU(),
    # Hidden
    nn.Linear(64, 64),
    # Activation
    nn.ReLU(),
    # Ouput
    nn.Linear(64, 3),
    # Final activation
    nn.Softmax(dim=1),
)
learning_rate, epochs = 0.01, 1_000
# Optimiser
optimiser = torch.optim.SGD(params=clf.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
losses = torch.zeros(size=(epochs, 1))
accuracy_list = []

# Train model
for epoch_idx in range(epochs):
    # Reset the gradients from prev. step loss.backward()
    optimiser.zero_grad()

    # Fwd prop
    _y_pred = clf(X)

    # Compute loss
    loss = criterion(_y_pred, y)
    losses[epoch_idx] = loss

    # Back prop
    loss.backward()
    optimiser.step()

    # Calculate accuracy
    y_p = torch.argmax(_y_pred, axis=1)
    acc_ = torch.mean((y_p == y).float())
    accuracy_list.append(acc_)

print("Training done ...")

Training done ...


In [6]:
# Make preductions
y_pred_raw = clf(X)

# Evaluate model
y_pred = torch.argmax(y_pred_raw, axis=1)
accuracy = torch.mean((y_pred == y).float())

accuracy

tensor(0.9600)